In [1]:
import os
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset
from azureml.data.datapath import DataPath
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.conda_dependencies import CondaDependencies

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.39.0


### Prihlasenie do azure

In [2]:
ws = Workspace.from_config('azure_config.json', auth=InteractiveLoginAuthentication())
print(ws.name, ws.location, ws.resource_group, sep='\t')

a0047stufiitml01	westeurope	a0047-STUFIIT-ML01


### Nacitanie datasetu

In [3]:
dataset = Dataset.get_by_name(workspace=ws, name='BraTS-dataset')

In [4]:
# nefunguje :(
#dataset.to_path()[0]

## Experimenty

In [5]:
config = ScriptRunConfig(source_directory='../src/',
                         script='main.py',
                         arguments=['--wandb', "a68b79c89593d1f8c50631508f2e618f3ba7ccf3",
                                    '--data_path', dataset.as_mount()], # This is important how to mount dataset from DataStore
                         compute_target='P100x1-xpovinec') # Compute target is your created compute cluster


In [6]:
experiment = Experiment(workspace=ws, name='xpovinec-test')

### Nastavenie env

In [7]:
# toto vypise vsetky env
for i,key in enumerate(ws.environments.keys()):
    print(f"{i}.",key)

0. pt-wandb-env
1. tf-wandb-env
2. Torch1.8-Opencv-wandb
3. torch-machata
4. buban-env
5. tf-gpu-sakalik
6. torch-martinkovic
7. kuran-env
8. tf-opencv-keras
9. Monai-Wandb-Cv
10. Torch1.8-Pillow-wandb
11. krajkovic-env
12. kabac-env
13. Halinkovic-Unet-env
14. Popovcova-torch-wandb
15. xpovinec-tf
16. xpovinec-env
17. slimak-env
18. xfranczel-env
19. xzhan-env
20. Salko-Torch
21. AzureML-Triton
22. AzureML-mlflow-ubuntu18.04-py37-cpu-inference
23. AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference
24. AzureML-minimal-ubuntu18.04-py37-cpu-inference
25. AzureML-tensorflow-2.4-ubuntu18.04-py37-cpu-inference
26. AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11.0.3-gpu-inference
27. AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
28. AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
29. AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
30. AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu
31. AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
32. AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu
33. AzureML-p

In [23]:
ws.environments

{'pt-wandb-env': {
     "databricks": {
         "eggLibraries": [],
         "jarLibraries": [],
         "mavenLibraries": [],
         "pypiLibraries": [],
         "rcranLibraries": []
     },
     "docker": {
         "arguments": [],
         "baseDockerfile": null,
         "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04",
         "baseImageRegistry": {
             "address": null,
             "password": null,
             "registryIdentity": null,
             "username": null
         },
         "enabled": true,
         "platform": {
             "architecture": "amd64",
             "os": "Linux"
         },
         "sharedVolumes": true,
         "shmSize": null
     },
     "environmentVariables": {
         "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
     },
     "inferencingStackVersion": null,
     "name": "pt-wandb-env",
     "python": {
         "baseCondaEnvironment": null,
         "condaDependencies": {
             "channels": [
     

In [19]:
# vytvorenie noveho env
conda = CondaDependencies()

# add channels
#conda.add_channel('pytorch')

# add conda packages
conda.add_conda_package('pip=20.2.4')
conda.add_conda_package('python=3.8')

# add pip packages
conda.add_pip_package('keras~=2.6.0')
#conda.add_pip_package('Keras-Preprocessing~=2.8.0')
conda.add_pip_package('matplotlib~=3.5.1')
conda.add_pip_package('nibabel~=3.2.2')
conda.add_pip_package('numpy~=1.21.3')
conda.add_pip_package('pandas~=1.4.1')
conda.add_pip_package('scikit-learn~=1.0.2')
conda.add_pip_package('scipy~=1.7.0')
conda.add_pip_package('tensorflow-gpu~=2.7.0')
conda.add_pip_package('azureml-core~=1.39.0')
conda.add_pip_package('opencv-python~=4.5.5.64')
conda.add_pip_package('opencv-python-headless~=4.5.5.64')
conda.add_pip_package('scikit-image')
conda.add_pip_package('segmentation-models-3D')

# create environment
env = Environment('xpovinec-env')
env.python.conda_dependencies = conda
env.save_to_directory('../', overwrite=True)

In [7]:
env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220208.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "xpovinec-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
 

In [62]:
# vytvorenie noveho env z requirements.txt
#env = Environment.from_pip_requirements(name='xpovinec-env', file_path='../requirements.txt')

In [20]:
# pridanie na azure, ked zmenim nejaky package tak sa updatne
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220208.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "xpovinec-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
 

In [18]:
# nieco s dockerom, neviem ci to treba davat vzdy
azureml.core.runconfig.DockerConfiguration(use_docker=True)

In [7]:
# vyber env
env = Environment.get(workspace=ws, name='xpovinec-env')
env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "xpovinec-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forg

In [8]:
# nastavenie env pre ten experiment
config.run_config.environment = env

### Spustenie experimentu

In [9]:
run = experiment.submit(config)

In [ ]:
# toto zisti, kde bezi a vrati to stranku
aml_url = run.get_portal_url()
print(aml_url)